# ROS messaging modules

We’ve processed all the standard ROS messages with our code generation tool creating a collection of strongly-typed IML/OCaml bindings for them. Let us load the relevant modules into Imandra:

In [1]:
System.mod_use ~quiet:true "src-messages/basic_types.ml";;
System.mod_use ~quiet:true "src-messages/std_msgs.ml";;
System.mod_use ~quiet:true "src-messages/rosgraph_msgs.ml";;
System.mod_use ~quiet:true "src-messages/geometry_msgs.ml";;
System.mod_use ~quiet:true "src-messages/sensor_msgs.ml";;

- : unit = ()
- : unit = ()
- : unit = ()
- : unit = ()
- : unit = ()


# State datatype

Working with Imandra we’ve adopted a standard way to construct formal models of message-driven systems. At the top of the model we have a single OCaml datatype that holds all the data needed to describe the system at a given moment, including incoming and outgoing messages. We call this record type `state`. Together with this `state` type we define a `one_step` transition `state -> state` function, which performs a single logically isolated step of the simulation and returns the new `state` after the transition.


As an example, consider an IML/OCaml type declaration for a simple ROS node that is able to accept `rosgraph_msgs/Clock` and `sensor_msgs/LaserScan` standard ROS messages. We also want the state to store the latest minimal value of the ranges that the laser sensor returns. And, finally, we want the node to be able to send `geometry_msgs/Twist` ROS message depending on the stored `min_range` data:

In [2]:
type incoming_msg = 
  | Clock  of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan

type outgoing_msg =
  | Twist of Geometry_msgs.twist

type state =
  { min_range : Basic_types.float64 option
  ; incoming  : incoming_msg option
  ; outgoing  : outgoing_msg option 
  }

type incoming_msg =
    Clock of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan
type outgoing_msg = Twist of Geometry_msgs.twist
type state = {
  min_range : Basic_types.float64 option;
  incoming : incoming_msg option;
  outgoing : outgoing_msg option;
}


The initial state of our model doesn't have any incoming messages nor any stored value:  

In [3]:
let init_state = 
  { min_range = None
  ; incoming  = None
  ; outgoing  = None 
  }

val init_state : state = {min_range = None; incoming = None; outgoing = None}


# State transition `one_step` function

Lets now create a simple `one_step` function for a ROS node that reacts to the incoming messages. We want the robot, on incoming `Clock` message, to either move forward or turn clockwise depending on the stored `min_range` value. To this end, we first create two helper functions: 
 - `get_min_range` extract the minimal value of the ranges in a `Scan` message 
 - `make_twist_message` creates a `Twist` message with the given forward and angular velocities

In [4]:
let get_min_range msg : Basic_types.float64 = 
  let min = List.fold_left (fun a b -> if a < b then a else b) 0 in
  min msg.Sensor_msgs.laserScan_ranges
  
let make_twist_message v omega=
  let open Geometry_msgs in
  let mkvector x y z = { vector3_x = x; vector3_y = y; vector3_z = z } in 
  Twist { twist_linear  = mkvector v 0 0
        ; twist_angular = mkvector 0 0 omega 
        } 

val get_min_range : Sensor_msgs.laserScan -> Basic_types.float64 = <fun>
val make_twist_message :
  Basic_types.float64 -> Basic_types.float64 -> outgoing_msg = <fun>


With the help of these functions, we can create our `one_step` transition:

In [5]:
let one_step state =
  match state.incoming with None -> state | Some in_msg ->
  let state = { state with incoming = None; outgoing = None } in
  match in_msg with 
  | Sensor laserScan -> 
    { state with min_range = Some (get_min_range laserScan) }
  | Clock  _ -> begin
    match state.min_range with None -> state | Some min_range ->
    if min_range < 50000
    then { state with outgoing = Some (make_twist_message 0 10000) } 
    else { state with outgoing = Some (make_twist_message 10000 0) }
  end

val one_step : state -> state = <fun>


# Running the model as a ROS node

One can write the model into an OCaml code file:   
https://github.com/AestheticIntegration/imandra-ros/blob/master/imandra_model/src-model/ros_model.ml  
and then compile it into an executable node, using our ROS node wrapper. 

Here is the model, controling our "imandrabot" in the Gazebo simulation environment:

![Imandrabot](../docs/images/Imandrabot.gif)